<a href="https://colab.research.google.com/github/andersonfmcz/caeo2025/blob/main/todas_regioes_enem2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Importação das bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import interact, widgets
from IPython.display import display
from IPython.display import display, HTML
from google.colab import drive
from IPython.display import clear_output


In [9]:
# Base de dados reduzida do Enem 2023
link_base = 'https://media.githubusercontent.com/media/atc857/enem2023/refs/heads/main/enem2023.csv'
enem2023 = pd.read_csv(link_base, encoding='ISO-8859-1')

In [10]:
# Mapeamento de UF para Região
uf_para_regiao = {
    'Norte': ['AC', 'AM', 'AP', 'PA', 'RO', 'RR', 'TO'],
    'Nordeste': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    'Centro-Oeste': ['DF', 'GO', 'MT', 'MS'],
    'Sudeste': ['ES', 'MG', 'RJ', 'SP'],
    'Sul': ['PR', 'RS', 'SC']
}
#drive.mount('/content/drive')# Criar coluna 'Região' no DataFrame
enem2023['REGIAO'] = enem2023['SG_UF_PROVA'].apply(
    lambda uf: next((regiao for regiao, ufs in uf_para_regiao.items() if uf in ufs), 'Outro')
)

# Criar o Combo Box com as regiões
dropdown_regioes = widgets.Dropdown(
    options=list(uf_para_regiao.keys()),
    description='Região:',
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)

# Função para filtrar e mostrar dados
def filtrar_por_regiao(regiao):
    ufs = uf_para_regiao[regiao]
    enem_norte = enem2023[enem2023['SG_UF_PROVA'].isin(ufs)]

    display(HTML("<h1 style='font-weight:bold; font-size:18px; color:#FF5733;'>Questão 1</h1>"))

    # Carregando e exibindo as útlimas linhas do dataframe.
    print("\nAs 5 primeiras linhas da região %s:" %(regiao))
    display(enem_norte.head())

    # Carregando e exibindo as útlimas linhas do dataframe.
    print("\nAs 5 últimas linhas da região %s:" %(regiao))
    display(enem_norte.tail())

    # Gerando e exibindo as dimensões (número de linhas e colunas) do dataframe.
    print("\nAs dimensões, número de linhas e colunas do dataframe:")
    display(enem_norte.shape)

    display(HTML("<h1 style='font-weight:bold; font-size:18px; color:#FF5733;'>Questão 2</h1>"))

    # Selecionando apenas as colunas de notas
    notas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

    # Excluindo valores nulos
    enem_norte = enem2023[enem2023['SG_UF_PROVA'].isin(ufs)][notas].dropna()

    # Gerando resumo estatístico
    resumo = enem_norte[notas].describe().loc[['min', '25%', 'max']]

    # Renomeando as linhas para melhor legibilidade
    resumo = resumo.rename(index={
        'min': 'Nota Mínima',
        '25%': 'Primeiro Quartil (Q1)',
        'max': 'Nota Máxima'
    })

    # Arredondando os valores para 2 casas decimais
    resumo = resumo.round(2)

    # Exibindo o resumo
    print("Resumo Estatístico das Notas da Região %s:" %(regiao))
    print(resumo)

    display(HTML("<h1 style='font-weight:bold; font-size:18px; color:#FF5733;'>Questão 3</h1>"))

    # Selecionando apenas as colunas de notas das provas
    provas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

    # Excluindo valores nulos
    enem_norte = enem2023[enem2023['SG_UF_PROVA'].isin(ufs)][provas].dropna()

    # Contando notas zero (0.0) em cada prova (total regional)
    notas_zero = (enem_norte[provas] == 0.0).sum().to_frame().T

    # Renomeando colunas e acrescentando o título TOTAL na última linha
    notas_zero.columns = [col.replace('NU_NOTA_', '') for col in provas]
    notas_zero.index = ['TOTAL']

    # Exibindo resultado
    print("Total de candidatos com nota zero (0.0) por prova.")
    print(notas_zero.to_string())

    display(HTML("<h1 style='font-weight:bold; font-size:18px; color:#FF5733;'>Questão 4</h1>"))

    enem_norte = enem2023[enem2023['SG_UF_PROVA'].isin(ufs)]
    nota_sem_zero_lc = enem_norte[(enem_norte['NU_NOTA_LC'] > 0.0) & (enem_norte['NU_NOTA_LC'].notna())]
    # Gerando o histograma da região Norte com nota zero em LC
    sns.histplot(data=nota_sem_zero_lc, x='NU_NOTA_LC', bins=20, kde=True)
    plt.title('Histograma das Notas de Linguagens e Códigos (Sem Notas Zero) da Região %s:' %(regiao))
    plt.xlabel('Notas de LC da Região %s (Sem Notas Zero)' %(regiao))
    plt.ylabel('Frequência')
    plt.show()

    display(HTML("<h1 style='font-weight:bold; font-size:18px; color:#FF5733;'>Questão 5</h1>"))

    enem_norte = enem2023[enem2023['SG_UF_PROVA'].isin(ufs)]
    # Contando participantes por município
    total_por_municipio = enem_norte['NO_MUNICIPIO_PROVA'].value_counts().reset_index()
    total_por_municipio.columns = ['Município', 'Total_Participantes']

    # Conforme o dicionário de dados do ENEM, a questão Q008 avalia a existência de banheiro na residência,
    # onde foi considerado a resposta "A" que indica ausência. Assim, de acordo com comando da questão,
    # filtramos exclusivamente candidatos que declararam não possuir banheiro (resposta "A"),
    # excluindo registros com valores nulos (não respondidos) ou outras respostas.
    sem_banheiro = enem_norte[enem_norte['Q008'] == 'A'].dropna()
    contagem_sem_banheiro = sem_banheiro['NO_MUNICIPIO_PROVA'].value_counts().reset_index()
    contagem_sem_banheiro.columns = ['Município', 'Sem_Banheiro']

    # Juntando os dados e calcular proporção
    dados_municipios = pd.merge(total_por_municipio, contagem_sem_banheiro, on='Município', how='left')
    dados_municipios['Sem_Banheiro'] = dados_municipios['Sem_Banheiro'].fillna(0)
    dados_municipios['Proporção'] = (dados_municipios['Sem_Banheiro'] / dados_municipios['Total_Participantes']) * 100

    # Ordenando e pegar os 10 municípios com maior proporção
    top10 = dados_municipios.sort_values('Proporção', ascending=False).head(10)

    # Formatando a tabela de resultados
    resultado = top10[['Município', 'Proporção']].round(2)
    resultado['Proporção'] = resultado['Proporção'].astype(str) + '%'

    # Exibindo tabela
    print("\nOs 10 Municípios com Maior Proporção de Participantes sem Banheiro em Casa:")
    print(resultado.to_string(index=False))

    display(HTML("<h1 style='font-weight:bold; font-size:18px; color:#FF5733;'>Questão 6</h1>"))

    # Selecionando apenas as colunas de notas
    notas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

    # Excluindo valores nulos
    enem_norte = enem2023[enem2023['SG_UF_PROVA'].isin(ufs)][notas].dropna()

    # Criando boxplots, com critério padrão para outliers de 1.5 (whis)
    plt.figure(figsize=(12, 7))
    sns.boxplot(data=enem_norte[notas], orient='h', whis=1.5)
    plt.title('Distribuição das Notas do ENEM 2023 - Região %s' %(regiao), pad=20)
    plt.xlabel('Notas')
    plt.ylabel('Provas')
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Análise Quantitativa de Outliers
    print("\nContagem de Outliers por Prova (critério Intervalo Interquartil - IQR * 1.5):")
    for prova in notas:
        q1 = enem_norte[prova].quantile(0.25)
        q3 = enem_norte[prova].quantile(0.75)
        iqr = q3 - q1
        lim_inf = q1 - 1.5 * iqr
        lim_sup = q3 + 1.5 * iqr
        outliers = enem_norte[(enem_norte[prova] < lim_inf) | (enem_norte[prova] > lim_sup)][prova]
        print(f"{prova.replace('NU_NOTA_', ''):<10}: {len(outliers):>4} outliers ({outliers.min():.1f} a {outliers.max():.1f})")

    print("\n")
# Integrar o dropdown com a função
interact(filtrar_por_regiao, regiao=dropdown_regioes)

# Opcional: Mostrar o mapeamento completo
print("\n")
#print("\nℹ️ Mapeamento de UF para Região:")
#for regiao, ufs in uf_para_regiao.items():
#    print(f"{regiao}: {', '.join(ufs)}")

interactive(children=(Dropdown(description='Região:', layout=Layout(width='300px'), options=('Norte', 'Nordest…

**I - Discussão sobre Outliers e Impacto na Análise**

**Natureza dos Outliers no ENEM 2023:**

**Notas altas:** Geralmente representam alunos excepcionais. Por exemplo, de acordo com boxplot gerado, em Matemática percebeu-se candidatos com notas entre 800 e 900, quando a maioria estava entre 400 a 600.

**Notas baixas:** Pelo bloxplot gerado, percebeu-se, por exemplo, candidatos com notas de Redação de 0 a 80, onde a maioria estava entre 500 a 800. E isso podem indicar problemas como:
*   Abandono da prova
*   Dificuldades extremas
*   Erros de registro

**Impacto nas Análises:**

**Distorção de médias:** Um pequeno número de notas muito altas pode inflacionar a média.

**Viés em correlações:** Outliers podem criar falsas relações entre variáveis.

**Modelos estatísticos:** Em análises como regressão, outliers podem influenciar desproporcionalmente os resultados, levando a conclusões erradas.

**II - Sugestão de métodos para lidar com esses valores extremos**

**Remoção:** Excluir registros com outliers, especialmente se forem erros ou irrelevantes para a análise (ex.: notas zeradas por ausência). Deve-se ter cuidado para não perder informações válidas.

**Transformação:** Aplicar transformações como logaritmo ou escalonamento para reduzir o impacto dos valores extremos.

**Limitação (capping):** Substituir outliers por valores nos limites inferior ou superior calculados pelo método IQR.

**Análise robusta:** Usar métricas menos sensíveis a outliers, como mediana e intervalo interquartil, ou modelos robustos que minimizem sua influência.

**Investigação:** Analisar o contexto dos outliers (ex.: notas máximas em redação podem ser válidas e indicar excelência) antes de qualquer tratamento.

**Análise separada:** Estudar outliers como grupo distinto quando relevante.

**Múltiplas abordagens:** Comparar resultados com e sem tratamento de outliers.

**Documentação:** Sempre registrar decisões sobre tratamento de valores extremos.